<a href="https://colab.research.google.com/github/ameympatil/NLP/blob/main/Bidirectional_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download c/fake-news/data

404 - Not Found


In [6]:
! kaggle competitions download fake-news

100% 46.5M/46.5M [00:03<00:00, 22.5MB/s]
100% 46.5M/46.5M [00:03<00:00, 15.8MB/s]


In [7]:
! mkdir fakenews

In [8]:
! unzip fake-news.zip -d fakenews

Archive:  fake-news.zip
  inflating: fakenews/submit.csv     
  inflating: fakenews/test.csv       
  inflating: fakenews/train.csv      


In [9]:
import pandas as pd
df = pd.read_csv('/content/fakenews/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [13]:
df.shape

(18285, 5)

In [12]:
df=df.dropna()

In [14]:
## get Independent features
X = df.drop('label',axis=1)

In [15]:
y = df['label']

In [16]:
## Check weather dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.9.2'

In [22]:
from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [23]:
## Vocabulary Size
voc_size=5000

In [24]:
messages = X.copy()

In [25]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [33]:
messages.reset_index(inplace=True)

In [28]:
import nltk
import re
from nltk.corpus import stopwords

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review = review.lower()
  review = review.split()
      
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)  


In [35]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [38]:
## One Hot Encoding
onehot_repr = [one_hot(words,voc_size)for words in corpus]
print(onehot_repr)

[[86, 3748, 1297, 609, 3615, 2075, 1901, 657, 4201, 4776], [4266, 3602, 4363, 342, 4480, 4332, 3113], [3353, 4310, 4619, 3761], [1716, 230, 2863, 1567, 4538, 4737], [2310, 4480, 4641, 3175, 3707, 2456, 4480, 525, 4027, 545], [3684, 3202, 664, 482, 3851, 2949, 2417, 2554, 4861, 1627, 2235, 404, 1154, 3350, 3113], [2485, 2747, 711, 4319, 956, 3587, 3483, 4745, 3498, 3197, 4884], [2721, 423, 1067, 1268, 4382, 1452, 2949, 3193, 3498, 3197, 4884], [4383, 4237, 930, 2156, 3946, 4716, 4741, 4148, 2949, 4846], [2296, 1959, 4980, 3410, 791, 4696, 956, 492], [4061, 393, 961, 4362, 4948, 1221, 4990, 4999, 174, 4880, 4984], [1567, 1547, 3615, 4716, 2949, 4382], [820, 121, 4038, 4669, 3373, 1429, 1584, 1868, 532], [2925, 4618, 2104, 3520, 773, 3644, 2823, 3498, 3197, 4884], [3575, 3543, 506, 4030, 3473, 3498, 3197, 4884], [3400, 4485, 1917, 2289, 2953, 2527, 2075, 304, 3450, 4784], [812, 2348, 3602], [3596, 719, 2236, 4586, 2949, 3250, 2211, 3113], [671, 4655, 4363, 3794, 4389, 4815, 1749, 429, 245

In [40]:
## Word Embedding Layer
# Pre Padding
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  657, 4201, 4776],
       [   0,    0,    0, ..., 4480, 4332, 3113],
       [   0,    0,    0, ..., 4310, 4619, 3761],
       ...,
       [   0,    0,    0, ..., 3498, 3197, 4884],
       [   0,    0,    0, ..., 1716, 2129, 2158],
       [   0,    0,    0, ..., 3315, 2125, 4660]], dtype=int32)

In [41]:
## Model Creation
embedding_features = 40 #dimensions
model = Sequential()
model.add(Embedding(voc_size,embedding_features,input_length=sent_length))

model.add(Bidirectional(LSTM(100)))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [42]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
## Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [45]:
## Model training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 14s 11ms/step - loss: 0.2523 - accuracy: 0.8845 - val_loss: 0.1968 - val_accuracy: 0.9173
Epoch 2/10
383/383 [==============================] - 3s 9ms/step - loss: 0.1280 - accuracy: 0.9510 - val_loss: 0.2065 - val_accuracy: 0.9181
Epoch 3/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0804 - accuracy: 0.9715 - val_loss: 0.2440 - val_accuracy: 0.9150
Epoch 4/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0520 - accuracy: 0.9832 - val_loss: 0.2752 - val_accuracy: 0.9112
Epoch 5/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0336 - accuracy: 0.9890 - val_loss: 0.3369 - val_accuracy: 0.9142
Epoch 6/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0199 - accuracy: 0.9942 - val_loss: 0.3688 - val_accuracy: 0.9112
Epoch 7/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0133 - accuracy: 0.9967 - val_loss: 0.4282 - val_accuracy: 0.9135
Epoc

In [46]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [49]:
y = np.where(y_pred>0.5,1,0)

In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y)

array([[3118,  301],
       [ 234, 2382]])

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y)

0.9113504556752279

In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

